<h1>10장 텍스트 임베딩 모델 만들기</h1>
<i>임베딩 모델을 훈련하고 미세 튜닝하는 방법 살펴 보기</i>

<a href="https://github.com/rickiepark/handson-llm"><img src="https://img.shields.io/badge/GitHub%20Repository-black?logo=github"></a>
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/rickiepark/handson-llm/blob/main/chapter10.ipynb)

---

이 노트북은 <[핸즈온 LLM](https://tensorflow.blog/handson-llm/)> 책 10장의 코드를 담고 있습니다.

---

<a href="https://tensorflow.blog/handson-llm/">
<img src="https://tensorflow.blog/wp-content/uploads/2025/05/ed95b8eca688ec98a8_llm.jpg" width="350"/></a>

### [선택사항] - <img src="https://colab.google/static/images/icons/colab.png" width=100>에서 패키지 선택하기


이 노트북을 구글 코랩에서 실행한다면 다음 코드 셀을 실행하여 이 노트북에서 필요한 패키지를  설치하세요.

---

💡 **NOTE**: 이 노트북의 코드를 실행하려면 GPU를 사용하는 것이 좋습니다. 구글 코랩에서는 **런타임 > 런타임 유형 변경 > 하드웨어 가속기 > T4 GPU**를 선택하세요.

---

사용 패키지 버전

* datasets 4.0.0
* sentence-transformers 5.2.0
* mteb 2.6.4
* nltk 3.9.1

In [1]:
# 깃허브에서 위젯 상태 오류를 피하기 위해 진행 표시줄을 나타내지 않도록 설정합니다.
from transformers.utils import logging

logging.disable_progress_bar()

In [2]:
# mteb 2.6.4 버전은 의존성 때문에 pytorch 2.8.0으로 다운그레이드하므로 의존성을 무시하고 설치합니다.
%%capture
!pip install mteb --no-deps
!pip install pytrec_eval

## 임베딩 모델 만들기

### 데이터

In [3]:
from datasets import load_dataset

# GLUE에서 MNLI 데이터셋을 로드합니다.
# 0 = 수반, 1 = 중립, 2 = 모순
train_dataset = load_dataset("glue", "mnli", split="train").select(range(50_000))
train_dataset = train_dataset.remove_columns("idx")

Generating train split:   0%|          | 0/392702 [00:00<?, ? examples/s]

Generating validation_matched split:   0%|          | 0/9815 [00:00<?, ? examples/s]

Generating validation_mismatched split:   0%|          | 0/9832 [00:00<?, ? examples/s]

Generating test_matched split:   0%|          | 0/9796 [00:00<?, ? examples/s]

Generating test_mismatched split:   0%|          | 0/9847 [00:00<?, ? examples/s]

In [4]:
train_dataset[2]

{'premise': 'One of our number will carry out your instructions minutely.',
 'hypothesis': 'A member of my team will execute your orders with immense precision.',
 'label': 0}

### 모델

In [5]:
from sentence_transformers import SentenceTransformer

# BERT 베이스 모델을 사용합니다.
embedding_model = SentenceTransformer('bert-base-uncased')

### 손실 함수

In [6]:
from sentence_transformers import losses

# 손실 함수를 정의합니다. 소프트맥스 손실을 위해 명시적으로 레이블의 개수를 지정해야 합니다.
train_loss = losses.SoftmaxLoss(
    model=embedding_model,
    sentence_embedding_dimension=embedding_model.get_sentence_embedding_dimension(),
    num_labels=3
)

### 평가

In [7]:
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator

# STSB를 위해 임베딩 유사도 평가자를 만듭니다.
val_sts = load_dataset('glue', 'stsb', split='validation')
evaluator = EmbeddingSimilarityEvaluator(
    sentences1=val_sts["sentence1"],
    sentences2=val_sts["sentence2"],
    scores=[score/5 for score in val_sts["label"]],
    main_similarity="cosine",
    similarity_fn_names=["cosine", "euclidean", "manhattan", "dot"]
)

Generating train split:   0%|          | 0/5749 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1379 [00:00<?, ? examples/s]

### 훈련

In [8]:
from sentence_transformers.training_args import SentenceTransformerTrainingArguments

# 훈련 매개변수를 정의합니다.
args = SentenceTransformerTrainingArguments(
    output_dir="base_embedding_model",
    num_train_epochs=1,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=100,
    fp16=True,
    eval_steps=100,
    logging_steps=100,
    report_to=[]
)

In [9]:
from sentence_transformers.trainer import SentenceTransformerTrainer

# 임베딩 모델을 훈련합니다.
trainer = SentenceTransformerTrainer(
    model=embedding_model,
    args=args,
    train_dataset=train_dataset,
    loss=train_loss,
    evaluator=evaluator
)
trainer.train()

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

dataset = dataset.select_columns(['hypothesis', 'entailment', 'contradiction'])


Step,Training Loss
100,1.074100
200,0.941000
300,0.878800
400,0.838700
500,0.818600
600,0.830700
700,0.807700
800,0.791800
900,0.780200
1000,0.763200


TrainOutput(global_step=1563, training_loss=0.8097891169942806, metrics={'train_runtime': 143.0302, 'train_samples_per_second': 349.576, 'train_steps_per_second': 10.928, 'total_flos': 0.0, 'train_loss': 0.8097891169942806, 'epoch': 1.0})

In [10]:
# 훈련된 모델을 평가합니다.
evaluator(embedding_model)

{'pearson_cosine': 0.5641660554366812,
 'spearman_cosine': 0.6231446216556664,
 'pearson_euclidean': 0.6026579469293775,
 'spearman_euclidean': 0.6242938804548781,
 'pearson_manhattan': 0.6127601834247947,
 'spearman_manhattan': 0.6289093551662129,
 'pearson_dot': 0.5390562019759275,
 'spearman_dot': 0.571222826083557,
 'pearson_max': 0.6127601834247947,
 'spearman_max': 0.6289093551662129}

### MTEB

In [12]:
!pip install pytrec_eval

  Preparing metadata (setup.py) ... done
  Created wheel for pytrec_eval: filename=pytrec_eval-0.5-cp312-cp312-linux_x86_64.whl size=309355 sha256=5ca245c8fe24b1d76dc655285518e50bff7878bc9b59ded18f9a1a4769d0506f
  Stored in directory: /root/.cache/pip/wheels/c6/4a/9e/e17f9ea004e1c221bd0ff384732285211c4917b790d598ea51
Successfully built pytrec_eval


In [39]:
import mteb
from pprint import pprint

# 평가 작업을 선택합니다.
tasks = mteb.get_tasks(tasks=["Banking77Classification.v2"])

# 결과를 계산합니다.
results = mteb.evaluate(embedding_model, tasks)
for task_results in results:
    pprint(task_results.only_main_score().to_dict())

Evaluating tasks:   0%|          | 0/1 [00:00<?, ?it/s]

{'dataset_revision': '18072d2685ea682290f7b8924d94c62acc19c0b2',
 'evaluation_time': 52.51340866088867,
 'kg_co2_emissions': None,
 'mteb_version': '2.6.4',
 'scores': {'test': [{'hf_subset': 'default',
                      'languages': ['eng-Latn'],
                      'main_score': 0.61593}]},
 'task_name': 'Banking77Classification.v2'}


⚠️ **VRAM 비우기** - 다음 코드를 사용해 VRAM(GPU RAM)을 비우세요. 만약 비워지지 않으면 노트북을 재시작해야 합니다. 코랩을 사용하는 경우 오른쪽의 리소스 탭에서 VRAM이 줄어 들었는지 확인할 수 있습니다. 또는 `!nvidia-smi` 명령을 실행하여 현재 사용량을 확인할 수 있습니다.

In [40]:
import gc
import torch

gc.collect()
torch.cuda.empty_cache()

### 손실 함수

#### 코사인 유사도 손실

In [41]:
from datasets import Dataset, load_dataset

# GLUE로부터 MNLI 데이터셋을 로드합니다.
# 0 = 수반, 1 = 중립, 2 = 모순
train_dataset = load_dataset("glue", "mnli", split="train").select(range(50_000))
train_dataset = train_dataset.remove_columns("idx")

# (중립/모순)=0, (수반)=1
mapping = {2: 0, 1: 0, 0:1}
train_dataset = Dataset.from_dict({
    "sentence1": train_dataset["premise"],
    "sentence2": train_dataset["hypothesis"],
    "label": [float(mapping[label]) for label in train_dataset["label"]]
})

In [42]:
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator

# STSB를 위한 임베딩 유사도 평가자를 만듭니다.
val_sts = load_dataset('glue', 'stsb', split='validation')
evaluator = EmbeddingSimilarityEvaluator(
    sentences1=val_sts["sentence1"],
    sentences2=val_sts["sentence2"],
    scores=[score/5 for score in val_sts["label"]],
    main_similarity="cosine",
    similarity_fn_names=["cosine", "euclidean", "manhattan", "dot"]
)

In [43]:
from sentence_transformers import losses, SentenceTransformer
from sentence_transformers.trainer import SentenceTransformerTrainer
from sentence_transformers.training_args import SentenceTransformerTrainingArguments

# 모델
embedding_model = SentenceTransformer('bert-base-uncased')

# 손실 함수
train_loss = losses.CosineSimilarityLoss(model=embedding_model)

# 훈련 매개변수
args = SentenceTransformerTrainingArguments(
    output_dir="cosineloss_embedding_model",
    num_train_epochs=1,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=100,
    fp16=True,
    eval_steps=100,
    logging_steps=100,
    report_to=[]
)

# 모델 훈련
trainer = SentenceTransformerTrainer(
    model=embedding_model,
    args=args,
    train_dataset=train_dataset,
    loss=train_loss,
    evaluator=evaluator
)
trainer.train()

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss
100,0.229800
200,0.169300
300,0.171300
400,0.158500
500,0.152900
600,0.159100
700,0.152200
800,0.157100
900,0.149000
1000,0.144800


TrainOutput(global_step=1563, training_loss=0.1571077382190824, metrics={'train_runtime': 148.9529, 'train_samples_per_second': 335.677, 'train_steps_per_second': 10.493, 'total_flos': 0.0, 'train_loss': 0.1571077382190824, 'epoch': 1.0})

In [44]:
# 훈련된 모델을 평가합니다.
evaluator(embedding_model)

{'pearson_cosine': 0.7150237915414502,
 'spearman_cosine': 0.717207296805996,
 'pearson_euclidean': 0.7259427117389942,
 'spearman_euclidean': 0.7264670184904889,
 'pearson_manhattan': 0.7271349999052888,
 'spearman_manhattan': 0.7279521351724266,
 'pearson_dot': 0.6667695446068267,
 'spearman_dot': 0.6677248125236539,
 'pearson_max': 0.7271349999052888,
 'spearman_max': 0.7279521351724266}

⚠️ **VRAM 비우기**

In [45]:
import gc
import torch

gc.collect()
torch.cuda.empty_cache()

#### MNR 손실

In [47]:
import random
from tqdm import tqdm
from datasets import Dataset, load_dataset

# GLUE에서 MNLI 데이터셋을 로드합니다.
mnli = load_dataset("glue", "mnli", split="train").select(range(50_000))
mnli = mnli.remove_columns("idx")
mnli = mnli.filter(lambda x: True if x['label'] == 0 else False)

# 데이터를 준비합니다.
train_dataset = {"anchor": [], "positive": [], "negative": []}
soft_negatives = list(mnli["hypothesis"]) # 'Column' 객체를 list로 변환
random.shuffle(soft_negatives)
for row, soft_negative in tqdm(zip(mnli, soft_negatives)):
    train_dataset["anchor"].append(row["premise"])
    train_dataset["positive"].append(row["hypothesis"])
    train_dataset["negative"].append(soft_negative)
train_dataset = Dataset.from_dict(train_dataset)
len(train_dataset)

16875it [00:01, 13963.44it/s]


16875

In [48]:
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator

# STSB를 위해 임베딩 유사도 평가자를 만듭니다.
val_sts = load_dataset('glue', 'stsb', split='validation')
evaluator = EmbeddingSimilarityEvaluator(
    sentences1=val_sts["sentence1"],
    sentences2=val_sts["sentence2"],
    scores=[score/5 for score in val_sts["label"]],
    main_similarity="cosine",
    similarity_fn_names=["cosine", "euclidean", "manhattan", "dot"]
)

In [49]:
from sentence_transformers import losses, SentenceTransformer
from sentence_transformers.trainer import SentenceTransformerTrainer
from sentence_transformers.training_args import SentenceTransformerTrainingArguments

# 모델
embedding_model = SentenceTransformer('bert-base-uncased')

# 손실 함수
train_loss = losses.MultipleNegativesRankingLoss(model=embedding_model)

# 훈련 매개변수
args = SentenceTransformerTrainingArguments(
    output_dir="mnrloss_embedding_model",
    num_train_epochs=1,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=100,
    fp16=True,
    eval_steps=100,
    logging_steps=100,
    report_to=[]
)

# 모델 훈련
trainer = SentenceTransformerTrainer(
    model=embedding_model,
    args=args,
    train_dataset=train_dataset,
    loss=train_loss,
    evaluator=evaluator
)
trainer.train()

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss
100,0.319500
200,0.102600
300,0.084400
400,0.065800
500,0.070600


TrainOutput(global_step=528, training_loss=0.1250853459491874, metrics={'train_runtime': 73.5522, 'train_samples_per_second': 229.429, 'train_steps_per_second': 7.179, 'total_flos': 0.0, 'train_loss': 0.1250853459491874, 'epoch': 1.0})

In [50]:
# 훈련된 모델을 평가합니다.
evaluator(embedding_model)

{'pearson_cosine': 0.8073870942993062,
 'spearman_cosine': 0.8093790044608586,
 'pearson_euclidean': 0.8200554759867678,
 'spearman_euclidean': 0.815219131261802,
 'pearson_manhattan': 0.8203599201691107,
 'spearman_manhattan': 0.8154963147268641,
 'pearson_dot': 0.7443997376319675,
 'spearman_dot': 0.7324344656638817,
 'pearson_max': 0.8203599201691107,
 'spearman_max': 0.8154963147268641}

## 미세 튜닝

⚠️ **VRAM 비우기**

In [51]:
import gc
import torch

gc.collect()
torch.cuda.empty_cache()

### 지도 학습 방법

In [52]:
from datasets import load_dataset
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator

# GLUE에서 MNLI 데이터셋을 로드합니다.
# 0 = 수반, 1 = 중립, 2 = 모순
train_dataset = load_dataset("glue", "mnli", split="train").select(range(50_000))
train_dataset = train_dataset.remove_columns("idx")

# STSB를 위해 임베딩 유사도 평가자를 만듭니다.
val_sts = load_dataset('glue', 'stsb', split='validation')
evaluator = EmbeddingSimilarityEvaluator(
    sentences1=val_sts["sentence1"],
    sentences2=val_sts["sentence2"],
    scores=[score/5 for score in val_sts["label"]],
    main_similarity="cosine",
    similarity_fn_names=["cosine", "euclidean", "manhattan", "dot"]
)

In [53]:
from sentence_transformers import losses, SentenceTransformer
from sentence_transformers.trainer import SentenceTransformerTrainer
from sentence_transformers.training_args import SentenceTransformerTrainingArguments

# 모델
embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# 손실 함수
train_loss = losses.MultipleNegativesRankingLoss(model=embedding_model)

# 훈련 매개변수
args = SentenceTransformerTrainingArguments(
    output_dir="finetuned_embedding_model",
    num_train_epochs=1,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=100,
    fp16=True,
    eval_steps=100,
    logging_steps=100,
    report_to=[]
)

# 모델 훈련
trainer = SentenceTransformerTrainer(
    model=embedding_model,
    args=args,
    train_dataset=train_dataset,
    loss=train_loss,
    evaluator=evaluator
)
trainer.train()

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

dataset = dataset.select_columns(['hypothesis', 'entailment', 'contradiction'])


Step,Training Loss
100,0.160800
200,0.116300
300,0.127500
400,0.114000
500,0.109700
600,0.101400
700,0.116100
800,0.103700
900,0.107800
1000,0.101700


TrainOutput(global_step=1563, training_loss=0.110532766111524, metrics={'train_runtime': 83.0724, 'train_samples_per_second': 601.885, 'train_steps_per_second': 18.815, 'total_flos': 0.0, 'train_loss': 0.110532766111524, 'epoch': 1.0})

In [54]:
# 훈련된 모델을 평가합니다.
evaluator(embedding_model)

{'pearson_cosine': 0.8488630363102132,
 'spearman_cosine': 0.8483460044917779,
 'pearson_euclidean': 0.8519429099185869,
 'spearman_euclidean': 0.8483460044917779,
 'pearson_manhattan': 0.8509304816635406,
 'spearman_manhattan': 0.8471923757183873,
 'pearson_dot': 0.8488630308109899,
 'spearman_dot': 0.848346146976225,
 'pearson_max': 0.8519429099185869,
 'spearman_max': 0.848346146976225}

In [55]:
# 사전 훈련된 모델을 평가합니다.
original_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
evaluator(original_model)

{'pearson_cosine': 0.8696194614248246,
 'spearman_cosine': 0.8671631197908374,
 'pearson_euclidean': 0.8678716015126742,
 'spearman_euclidean': 0.8671631190200253,
 'pearson_manhattan': 0.8670399046828885,
 'spearman_manhattan': 0.8663946131522758,
 'pearson_dot': 0.8696194596369274,
 'spearman_dot': 0.8671631197908374,
 'pearson_max': 0.8696194614248246,
 'spearman_max': 0.8671631197908374}

⚠️ **VRAM 비우기**

In [56]:
import gc
import torch

gc.collect()
torch.cuda.empty_cache()

### 증식 SBERT

**단계 1:** 크로스 인코더를 미세 튜닝합니다.

In [57]:
import pandas as pd
from tqdm import tqdm
from datasets import load_dataset, Dataset
from sentence_transformers import InputExample
from sentence_transformers.datasets import NoDuplicatesDataLoader

# 크로스 인코더를 위해 10,000개의 문서로 구성된 데이터셋을 만듭니다.
dataset = load_dataset("glue", "mnli", split="train").select(range(10_000))
mapping = {2: 0, 1: 0, 0:1}

# 데이터 로더
gold_examples = [
    InputExample(texts=[row["premise"], row["hypothesis"]], label=mapping[row["label"]])
    for row in tqdm(dataset)
]
gold_dataloader = NoDuplicatesDataLoader(gold_examples, batch_size=32)

# 데이터 처리를 쉽게 하기 위해 판다스 데이터프레임을 만듭니다.
gold = pd.DataFrame(
    {
    'sentence1': dataset['premise'],
    'sentence2': dataset['hypothesis'],
    'label': [mapping[label] for label in dataset['label']]
    }
)

100%|██████████| 10000/10000 [00:00<00:00, 23808.78it/s]


In [61]:
from sentence_transformers.cross_encoder import CrossEncoder

# 골드 데이터셋에서 크로스 인코더를 훈련합니다.
cross_encoder = CrossEncoder('bert-base-uncased', num_labels=2)
cross_encoder.fit(
    train_dataloader=gold_dataloader,
    epochs=1,
    show_progress_bar=True,
    warmup_steps=100,
    use_amp=False
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss


**단계 2:** 새로운 문장 쌍을 만듭니다.

In [62]:
# 크로스 인코더로 레이블을 예측할 실버 데이터셋을 만듭니다.
silver = load_dataset("glue", "mnli", split="train").select(range(10_000, 50_000))
pairs = list(zip(silver['premise'], silver['hypothesis']))

**단계 3:** 미세 튜닝된 크로스 인코더로 새로운 문장 쌍(실버 데이터셋)에 레이블을 할당합니다.

In [63]:
import numpy as np

# 미세 튜닝된 크로스 인코더를 사용해 문장 쌍에 레이블을 할당합니다.
output = cross_encoder.predict(pairs, apply_softmax=True,
                               show_progress_bar=True)
silver = pd.DataFrame(
    {
        "sentence1": silver["premise"],
        "sentence2": silver["hypothesis"],
        "label": np.argmax(output, axis=1)
    }
)

Batches:   0%|          | 0/1250 [00:00<?, ?it/s]

**단계 4:** 확장된 데이터셋(골드 데이터셋 + 실버 데이터셋)으로 바이 인코더(SBERT)를 훈련합니다.

In [64]:
# 골드 데이터셋과 실버 데이터셋을 합칩니다.
data = pd.concat([gold, silver], ignore_index=True, axis=0)
data = data.drop_duplicates(subset=['sentence1', 'sentence2'], keep="first")
train_dataset = Dataset.from_pandas(data, preserve_index=False)

In [65]:
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator

# STSB를 위한 임베딩 유사도 평가자를 만듭니다.
val_sts = load_dataset('glue', 'stsb', split='validation')
evaluator = EmbeddingSimilarityEvaluator(
    sentences1=val_sts["sentence1"],
    sentences2=val_sts["sentence2"],
    scores=[score/5 for score in val_sts["label"]],
    main_similarity="cosine",
    similarity_fn_names=["cosine", "euclidean", "manhattan", "dot"]
)

In [66]:
from sentence_transformers import losses, SentenceTransformer
from sentence_transformers.trainer import SentenceTransformerTrainer
from sentence_transformers.training_args import SentenceTransformerTrainingArguments

# 모델
embedding_model = SentenceTransformer('bert-base-uncased')

# 손실 함수
train_loss = losses.CosineSimilarityLoss(model=embedding_model)

# 훈련 매개변수
args = SentenceTransformerTrainingArguments(
    output_dir="augmented_embedding_model",
    num_train_epochs=1,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=100,
    fp16=True,
    eval_steps=100,
    logging_steps=100,
    report_to=[]
)

# 모델 훈련
trainer = SentenceTransformerTrainer(
    model=embedding_model,
    args=args,
    train_dataset=train_dataset,
    loss=train_loss,
    evaluator=evaluator
)
trainer.train()

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss
100,0.218600
200,0.157700
300,0.143900
400,0.142900
500,0.138200
600,0.137400
700,0.134100
800,0.131900
900,0.134900
1000,0.130800


TrainOutput(global_step=1563, training_loss=0.140699625472876, metrics={'train_runtime': 152.9561, 'train_samples_per_second': 326.878, 'train_steps_per_second': 10.219, 'total_flos': 0.0, 'train_loss': 0.140699625472876, 'epoch': 1.0})

In [67]:
# 훈련된 모델을 평가합니다.
evaluator(embedding_model)

{'pearson_cosine': 0.7071762588925296,
 'spearman_cosine': 0.7126295249676218,
 'pearson_euclidean': 0.7101844683629198,
 'spearman_euclidean': 0.7080145146694184,
 'pearson_manhattan': 0.7121889709706318,
 'spearman_manhattan': 0.7099648131666315,
 'pearson_dot': 0.6530706891164939,
 'spearman_dot': 0.6563839346005123,
 'pearson_max': 0.7121889709706318,
 'spearman_max': 0.7126295249676218}

In [68]:
trainer.accelerator.clear()

[]

**단계 5**: 실버 데이터셋을 사용하지 않고 평가합니다.

In [69]:
# 골드 데이터셋만 사용합니다.
data = pd.concat([gold], ignore_index=True, axis=0)
data = data.drop_duplicates(subset=['sentence1', 'sentence2'], keep="first")
train_dataset = Dataset.from_pandas(data, preserve_index=False)

# 모델
embedding_model = SentenceTransformer('bert-base-uncased')

# 손실 함수
train_loss = losses.CosineSimilarityLoss(model=embedding_model)

# 훈련 매개변수
args = SentenceTransformerTrainingArguments(
    output_dir="gold_only_embedding_model",
    num_train_epochs=1,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=100,
    fp16=True,
    eval_steps=100,
    logging_steps=100,
    report_to=[]
)

# 모델 훈련
trainer = SentenceTransformerTrainer(
    model=embedding_model,
    args=args,
    train_dataset=train_dataset,
    loss=train_loss,
    evaluator=evaluator
)
trainer.train()

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss
100,0.227200
200,0.171900
300,0.160900


TrainOutput(global_step=313, training_loss=0.18587486279277374, metrics={'train_runtime': 30.9636, 'train_samples_per_second': 322.96, 'train_steps_per_second': 10.109, 'total_flos': 0.0, 'train_loss': 0.18587486279277374, 'epoch': 1.0})

In [70]:
# 훈련된 모델을 평가합니다.
evaluator(embedding_model)

{'pearson_cosine': 0.6455765400590219,
 'spearman_cosine': 0.670678910645092,
 'pearson_euclidean': 0.6531678908634108,
 'spearman_euclidean': 0.6658351688725158,
 'pearson_manhattan': 0.6544670203010571,
 'spearman_manhattan': 0.6673743710009832,
 'pearson_dot': 0.5758976179845045,
 'spearman_dot': 0.5811048636601204,
 'pearson_max': 0.6544670203010571,
 'spearman_max': 0.670678910645092}

실버 데이터셋과 골드 데이터셋을 모두 사용했을 때와 비교하면 골드 데이터셋만 사용한 경우 모델의 성능이 감소합니다!

⚠️ **VRAM 비우기**

In [71]:
import gc
import torch

gc.collect()
torch.cuda.empty_cache()

## 비지도 학습

### TSDAE

In [72]:
# 추가적인 토크나이저를 다운로드합니다.
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [74]:
from tqdm import tqdm
from datasets import Dataset, load_dataset
from sentence_transformers.datasets import DenoisingAutoEncoderDataset

# 전제와 가설을 하나의 문장으로 연결합니다.
mnli = load_dataset("glue", "mnli", split="train").select(range(25_000))
flat_sentences = list(mnli["premise"]) + list(mnli["hypothesis"])

# 입력 데이터에 잡음을 추가합니다.
damaged_data = DenoisingAutoEncoderDataset(list(set(flat_sentences)))

# 데이터셋을 만듭니다.
train_dataset = {"damaged_sentence": [], "original_sentence": []}
for data in tqdm(damaged_data):
    train_dataset["damaged_sentence"].append(data.texts[0])
    train_dataset["original_sentence"].append(data.texts[1])
train_dataset = Dataset.from_dict(train_dataset)

100%|██████████| 48353/48353 [00:07<00:00, 6432.54it/s]


In [75]:
train_dataset[0]

{'damaged_sentence': 'fact the uh someone come a seventy gotten the person the crime went someone there uh was on drugs and old was he for twelve',
 'original_sentence': 'in fact the news tonight they had uh someone come on a seventy three year old man they they had gotten into a housing for the elderly and and the person who perpetrated the crime went in with someone who had a relative there and uh apparently this this person was on drugs and the old man was an invalid and he was in an elevator and he died for twelve dollars'}

In [76]:
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator

# STSB를 위한 임베딩 유사도 평가자를 만듭니다.
val_sts = load_dataset('glue', 'stsb', split='validation')
evaluator = EmbeddingSimilarityEvaluator(
    sentences1=val_sts["sentence1"],
    sentences2=val_sts["sentence2"],
    scores=[score/5 for score in val_sts["label"]],
    main_similarity="cosine",
    similarity_fn_names=["cosine", "euclidean", "manhattan", "dot"]
)

In [77]:
from sentence_transformers import models, SentenceTransformer

# 임베딩 모델을 만듭니다.
word_embedding_model = models.Transformer('bert-base-uncased')
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(), 'cls')
embedding_model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

In [78]:
from sentence_transformers import losses

# 잡음제거 오토 인코더 손실
train_loss = losses.DenoisingAutoEncoderLoss(
    embedding_model, tie_encoder_decoder=True
)
train_loss.decoder = train_loss.decoder.to("cuda")

Some weights of BertLMHeadModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['bert.encoder.layer.0.crossattention.output.LayerNorm.bias', 'bert.encoder.layer.0.crossattention.output.LayerNorm.weight', 'bert.encoder.layer.0.crossattention.output.dense.bias', 'bert.encoder.layer.0.crossattention.output.dense.weight', 'bert.encoder.layer.0.crossattention.self.key.bias', 'bert.encoder.layer.0.crossattention.self.key.weight', 'bert.encoder.layer.0.crossattention.self.query.bias', 'bert.encoder.layer.0.crossattention.self.query.weight', 'bert.encoder.layer.0.crossattention.self.value.bias', 'bert.encoder.layer.0.crossattention.self.value.weight', 'bert.encoder.layer.1.crossattention.output.LayerNorm.bias', 'bert.encoder.layer.1.crossattention.output.LayerNorm.weight', 'bert.encoder.layer.1.crossattention.output.dense.bias', 'bert.encoder.layer.1.crossattention.output.dense.weight', 'bert.encoder.layer.1.crossattention.self.key.bias', 'bert.e

In [79]:
from sentence_transformers.trainer import SentenceTransformerTrainer
from sentence_transformers.training_args import SentenceTransformerTrainingArguments

# 훈련 매개변수
args = SentenceTransformerTrainingArguments(
    output_dir="tsdae_embedding_model",
    num_train_epochs=1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=100,
    fp16=True,
    eval_steps=100,
    logging_steps=100,
    report_to=[]
)

# 모델 훈련
trainer = SentenceTransformerTrainer(
    model=embedding_model,
    args=args,
    train_dataset=train_dataset,
    loss=train_loss,
    evaluator=evaluator
)
trainer.train()

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


Step,Training Loss
100,6.836700
200,4.850600
300,4.597300
400,4.444600
500,4.375600
600,4.259000
700,4.207100
800,4.114200
900,4.082500
1000,4.049000


TrainOutput(global_step=3023, training_loss=4.008500432636893, metrics={'train_runtime': 322.614, 'train_samples_per_second': 149.879, 'train_steps_per_second': 9.37, 'total_flos': 0.0, 'train_loss': 4.008500432636893, 'epoch': 1.0})

In [80]:
# 훈련된 모델을 평가합니다.
evaluator(embedding_model)

{'pearson_cosine': 0.7381847028261099,
 'spearman_cosine': 0.7455835405628096,
 'pearson_euclidean': 0.7369892684061766,
 'spearman_euclidean': 0.7399678623151376,
 'pearson_manhattan': 0.7366523707588917,
 'spearman_manhattan': 0.7400609495679075,
 'pearson_dot': 0.6372916059669801,
 'spearman_dot': 0.6382162388049192,
 'pearson_max': 0.7381847028261099,
 'spearman_max': 0.7455835405628096}